In [67]:
import pyspark
from pyspark.sql import SQLContext
from pyspark.sql.types import *
import pyspark.sql.functions as func
from pyspark.sql.functions import col
from pyspark.sql.functions import sum
from pyspark.sql.functions import when
import datetime

In [2]:
sc = pyspark.SparkContext(appName="Tags")
sqlContext = SQLContext(sc)

In [13]:
df = sqlContext.read.parquet("./data/results/Posts_parquet/Posts.parquet")

In [14]:
tags_df=df \
.select(df["Id"].cast('int'), 
        df["Tags"], 
        df["CreationDate"].cast('date'), 
        df['LastEditDate'].cast('date'))
tags_df.show(5)

+---+--------------------+------------+------------+
| Id|                Tags|CreationDate|LastEditDate|
+---+--------------------+------------+------------+
|  1|<mysql><innodb><m...|  2011-01-03|        null|
|  2|<mysql><version-c...|  2011-01-03|  2011-01-06|
|  3|<database-design>...|  2011-01-03|  2011-01-06|
|  4|                null|  2011-01-03|  2013-09-23|
|  5|<nosql><rdbms><da...|  2011-01-03|  2011-01-06|
+---+--------------------+------------+------------+
only showing top 5 rows



In [15]:
tag_dict = {
    "mysql.*": "mysql"
    ,"innodb": "mysql"
    ,"myisam": "mysql"
    ,"sql-server.*": "sql-server"
    ,"ssms": "sql-server"
    ,"ssis": "sql-server"
    ,"t-sql": "sql-server"
    ,"postgresql.*": "postgres"
    ,"oracle.*": "oracle"
    ,"document-oriented":"mongodb"
    ,"phpmyadmin": "postgres"
    ,"plsql": "oracle"
    ,"plpgsql": "postgres"
    ,"sqlplus": "oracle"
    ,"rman": "oracle"
    ,"pgadmin": "postgres"
    ,"windows-server.*":"sql-server"
    ,"pg-dump": "postgres"
    ,"psql": "postgres"
    ,"postgis": "postgres"
    ,"nosql": "mongodb"
    ,"mongo.*": "mongodb"
        }

In [16]:
def standardize_tags(tag_list):
#     tags_std=[tag.replace('innodb', 'mysql') for tag in tag_list]
    import re
    tags_std = tag_list
    for key, value in tag_dict.items():
        regex = re.compile(key, re.IGNORECASE)
        tags_std=[regex.sub(value, tag) for tag in tags_std]
#     for i in tag_dict:
#         regex = re.compile(i, re.IGNORECASE)
#         tags_std=[regex.sub(i% tag_dict[i], tag) for tag in tag_list]
    return tags_std

In [17]:
tags_rdd=tags_df.rdd
tags_rdd.count()

127212

In [23]:
tags_not_empty=tags_rdd.filter(lambda x: x[1]!=None)
tags_not_empty.count()

53664

In [28]:
tags_clean=tags_not_empty \
.map(lambda x: (x[0], x[1].replace('><',', '), x[2], x[3])) \
.map(lambda x: (x[0], x[1].replace('<', ''), x[2], x[3])) \
.map(lambda x: (x[0], x[1].replace('>', ''), x[2], x[3]))

tags_split=tags_clean \
.map(lambda x: (x[0], x[1].split(','), x[2], x[3])) \
.map(lambda x: (x[0], [e.strip() for e in x[1]], x[2], x[3]))

tags_standardized=tags_split \
.map(lambda x: (x[0], standardize_tags(x[1]), x[2], x[3]))

tags_unique = tags_standardized \
.map(lambda x: (x[0],set(x[1]), x[2], x[3]))

tags_unique.take(10)

[(1, {'mysql'}, datetime.date(2011, 1, 3), None),
 (2,
  {'mysql', 'schema', 'version-control'},
  datetime.date(2011, 1, 3),
  datetime.date(2011, 1, 6)),
 (3,
  {'database-design', 'erd'},
  datetime.date(2011, 1, 3),
  datetime.date(2011, 1, 6)),
 (5,
  {'database-recommendation', 'mongodb', 'rdbms'},
  datetime.date(2011, 1, 3),
  datetime.date(2011, 1, 6)),
 (6,
  {'postgres', 'replication'},
  datetime.date(2011, 1, 3),
  datetime.date(2011, 8, 16)),
 (14,
  {'database-recommendation', 'mariadb', 'mysql'},
  datetime.date(2011, 1, 3),
  datetime.date(2011, 8, 16)),
 (20,
  {'mysql', 'perfooracle'},
  datetime.date(2011, 1, 3),
  datetime.date(2011, 1, 6)),
 (21, {'sqlite'}, datetime.date(2011, 1, 3), datetime.date(2011, 8, 16)),
 (29,
  {'learning', 'sql'},
  datetime.date(2011, 1, 3),
  datetime.date(2015, 4, 23)),
 (30,
  {'postgres', 'trigger'},
  datetime.date(2011, 1, 3),
  datetime.date(2011, 12, 28))]

In [29]:
unique_tags=set(tag_dict.values())
unique_tags

{'mongodb', 'mysql', 'oracle', 'postgres', 'sql-server'}

In [37]:
post_tags_meaningful = tags_unique \
.map(lambda x: (x[0],x[1].intersection(unique_tags), x[2], x[3])) \
.filter(lambda x: bool(x[1]))
post_tags_meaningful.take(20)

[(1, {'mysql'}, datetime.date(2011, 1, 3), None),
 (2, {'mysql'}, datetime.date(2011, 1, 3), datetime.date(2011, 1, 6)),
 (5, {'mongodb'}, datetime.date(2011, 1, 3), datetime.date(2011, 1, 6)),
 (6, {'postgres'}, datetime.date(2011, 1, 3), datetime.date(2011, 8, 16)),
 (14, {'mysql'}, datetime.date(2011, 1, 3), datetime.date(2011, 8, 16)),
 (20, {'mysql'}, datetime.date(2011, 1, 3), datetime.date(2011, 1, 6)),
 (30, {'postgres'}, datetime.date(2011, 1, 3), datetime.date(2011, 12, 28)),
 (33, {'mysql'}, datetime.date(2011, 1, 3), datetime.date(2011, 9, 28)),
 (36, {'mysql'}, datetime.date(2011, 1, 3), datetime.date(2011, 8, 16)),
 (43, {'mysql'}, datetime.date(2011, 1, 3), datetime.date(2016, 11, 9)),
 (47, {'sql-server'}, datetime.date(2011, 1, 3), None),
 (49, {'mongodb'}, datetime.date(2011, 1, 3), datetime.date(2011, 8, 14)),
 (56, {'sql-server'}, datetime.date(2011, 1, 3), datetime.date(2013, 2, 22)),
 (63, {'mysql'}, datetime.date(2011, 1, 3), datetime.date(2011, 8, 16)),
 (64, {'

In [52]:
post_tags_meaningful.filter(lambda x: x[3] == None).count()

19002

In [38]:
common_tags = post_tags_meaningful.filter(lambda x: len(x[1])>1)
common_tags.take(20)

[(96,
  {'mysql', 'postgres'},
  datetime.date(2011, 1, 4),
  datetime.date(2011, 8, 16)),
 (287, {'oracle', 'sql-server'}, datetime.date(2011, 1, 5), None),
 (297,
  {'oracle', 'sql-server'},
  datetime.date(2011, 1, 5),
  datetime.date(2011, 1, 10)),
 (409,
  {'oracle', 'sql-server'},
  datetime.date(2011, 1, 7),
  datetime.date(2014, 4, 23)),
 (549, {'mysql', 'postgres'}, datetime.date(2011, 1, 11), None),
 (688,
  {'mysql', 'sql-server'},
  datetime.date(2011, 1, 18),
  datetime.date(2011, 8, 19)),
 (785,
  {'mysql', 'oracle'},
  datetime.date(2011, 1, 22),
  datetime.date(2011, 1, 22)),
 (830,
  {'mysql', 'sql-server'},
  datetime.date(2011, 1, 24),
  datetime.date(2011, 8, 15)),
 (831,
  {'mysql', 'sql-server'},
  datetime.date(2011, 1, 24),
  datetime.date(2011, 1, 27)),
 (1099, {'mysql', 'postgres'}, datetime.date(2011, 2, 6), None),
 (1322,
  {'oracle', 'sql-server'},
  datetime.date(2011, 2, 18),
  datetime.date(2011, 2, 21)),
 (1339, {'mysql', 'postgres'}, datetime.date(2011

In [76]:
post_tags_to_write=post_tags_meaningful \
.map(lambda x: (x[0],list(x[1]), x[2], x[3]))

fields = [
    StructField('Id', IntegerType(), True),
    StructField('Tags', StringType(), True),
    StructField('CreationDate', DateType(), True),
    StructField('LastEditDate', DateType(), True)]
schema = StructType(fields)
df=sqlContext.createDataFrame(post_tags_to_write, schema, samplingRatio = 0.1)
df_filled = df \
.withColumn('LastEditDate', when(
        col('LastEditDate').isNull(), datetime.date.today()).otherwise(col('LastEditDate')))
df_filled.show()

+---+------------+------------+------------+
| Id|        Tags|CreationDate|LastEditDate|
+---+------------+------------+------------+
|  1|     [mysql]|  2011-01-03|  2017-05-26|
|  2|     [mysql]|  2011-01-03|  2011-01-06|
|  5|   [mongodb]|  2011-01-03|  2011-01-06|
|  6|  [postgres]|  2011-01-03|  2011-08-16|
| 14|     [mysql]|  2011-01-03|  2011-08-16|
| 20|     [mysql]|  2011-01-03|  2011-01-06|
| 30|  [postgres]|  2011-01-03|  2011-12-28|
| 33|     [mysql]|  2011-01-03|  2011-09-28|
| 36|     [mysql]|  2011-01-03|  2011-08-16|
| 43|     [mysql]|  2011-01-03|  2016-11-09|
| 47|[sql-server]|  2011-01-03|  2017-05-26|
| 49|   [mongodb]|  2011-01-03|  2011-08-14|
| 56|[sql-server]|  2011-01-03|  2013-02-22|
| 63|     [mysql]|  2011-01-03|  2011-08-16|
| 64|[sql-server]|  2011-01-03|  2015-12-26|
| 66|  [postgres]|  2011-01-03|  2011-01-04|
| 71|[sql-server]|  2011-01-04|  2016-11-22|
| 80|    [oracle]|  2011-01-04|  2011-01-04|
| 81|    [oracle]|  2011-01-04|  2011-01-04|
| 82|    [

In [77]:
df_filled.coalesce(1).write.parquet("data/results/Posts_clean_parquet")

In [78]:
df1 = sqlContext.read.parquet("./data/results/Posts_clean_parquet/Posts.parquet")
# tags_df=df \
# .select(df["Id"].cast('int'), 
#         df["Tags"], 
#         df["CreationDate"].cast('date'), 
#         df['LastEditDate'].cast('date'))
df1.show(5)

+---+----------+------------+------------+
| Id|      Tags|CreationDate|LastEditDate|
+---+----------+------------+------------+
|  1|   [mysql]|  2011-01-03|  2017-05-26|
|  2|   [mysql]|  2011-01-03|  2011-01-06|
|  5| [mongodb]|  2011-01-03|  2011-01-06|
|  6|[postgres]|  2011-01-03|  2011-08-16|
| 14|   [mysql]|  2011-01-03|  2011-08-16|
+---+----------+------------+------------+
only showing top 5 rows

